In [ ]:
!pip install -q datasets transformers sentencepiece torch tqdm numpy
# ============================================================
# CELL 1: Setup & Dependencies
# ============================================================
!git clone https://github.com/auralmn/aura-hybrid-pre-model.git

# Uninstall existing torch and torchvision to prevent conflicts
# !pip uninstall -y torch torchvision torchaudio

# Explicitly install specific compatible versions first to prevent conflicts
# !pip install torchvision

# Install project dependencies, which should now recognize the already installed compatible torch/torchvision
!cd aura-hybrid-pre-model && git checkout master && git pull
import sys
sys.path.insert(0, '/content/aura-hybrid-pre-model')

# Import modules
from src.core.hippocampal import HippocampalFormation
from src.core.language_zone.hippocampal_transformer import HippocampalTransformer
from src.training.hippocampal_trainer import HippocampalTransformerTrainer

# Explicitly reload hippocampal_trainer to ensure latest changes are picked up
import importlib
importlib.reload(sys.modules['src.training.hippocampal_trainer'])

In [ ]:
import os
import gc
import math
import time
import traceback
from dataclasses import dataclass
from typing import Optional

import numpy as np
import torch
import torch.nn as nn
from torch.amp import autocast
from tqdm import tqdm

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

CHECKPOINT_DIR = '/content/drive/MyDrive/aura_checkpoints'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

device = torch.device('cuda')
print(f"✅ Device: {device}")
print(f"   GPU: {torch.cuda.get_device_name(0)}")
print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
print(f"   Checkpoint dir: {CHECKPOINT_DIR}")


In [ ]:
from dataclasses import dataclass


@dataclass
class Config:
    # === MODEL (L4 Optimized) ===
    vocab_size: int = 32000
    embedding_dim: int = 768
    num_layers: int = 12
    num_heads: int = 16
    head_dim: int = 64
    dropout: float = 0.15
    max_seq_len: int = 512
    intermediate_size: int = 4096

    # === HIPPOCAMPAL ===
    theta_frequency: float = 8.0
    gamma_frequency: float = 40.0
    n_place_cells: int = 2000
    n_time_cells: int = 100
    n_grid_cells: int = 200

    # === TRAINING ===
    batch_size: int = 16
    gradient_accumulation: int = 1
    lr: float = 3e-4
    warmup_steps: int = 1500
    max_steps: int = 50000
    weight_decay: float = 0.1

    # === CONSOLIDATION ===
    sleep_interval: int = 2000
    sleep_steps: int = 25
    eval_interval: int = 100
    ewc_lambda: float = 0.4
    use_ewc: bool = True

    # === MEMORY ===
    replay_buffer_size: int = 1000000
    memory_creation_interval: int = 5
    memory_decay_rate: float = 0.03

    # === TRAINING STABILITY ===
    label_smoothing: float = 0.2
    use_mixed_precision: bool = True
    compile_model: bool = False


config = Config()

print("="*60)
print("L4 CONFIG (22.5GB VRAM)")
print("="*60)
print(f"Model: {config.embedding_dim}D × {config.num_layers}L × {config.num_heads}H")
print(f"Batch: {config.batch_size} × {config.gradient_accumulation} = {config.batch_size * config.gradient_accumulation} effective")
print(f"LR: {config.lr} | Label smoothing: {config.label_smoothing}")
print(f"EWC: enabled (λ={config.ewc_lambda})")
print(f"Max steps: {config.max_steps}")
print("="*60)


In [ ]:
from datasets import load_dataset
from transformers import T5Tokenizer

import gc
import torch
from torch.amp import autocast

gc.collect()
torch.cuda.empty_cache()


print("="*60)
print("LOADING DATASET & TOKENIZER")
print("="*60)

print("\n📚 Loading Nemotron-CC-v2 High-Quality...")
try:
    dataset = load_dataset(
        "nvidia/Nemotron-CC-v2",
        "High-Quality",
        split="train",
        streaming=True
    )
    print("✅ Nemotron-CC-v2 loaded (streaming)")

except Exception as e:
    print(f"⚠️ Nemotron failed: {e}")
    print("Falling back to WikiText-103...")
    dataset = load_dataset('wikitext', 'wikitext-103-raw-v1', split='train')
    print("✅ WikiText-103 loaded")

# Load T5 Tokenizer
print("\n🔤 Loading T5 Tokenizer...")
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
sp = tokenizer.sp_model

print(f"✅ T5 Tokenizer loaded")
print(f"   Vocab size: {tokenizer.vocab_size}")
print(f"   Pad token: {tokenizer.pad_token_id}")

# Test
test_text = "The quick brown fox"
test_ids = sp.encode(test_text, out_type=int)
print(f"\n   Test: '{test_text}'")
print(f"   Tokens: {test_ids}")
print(f"   Decoded: '{sp.decode(test_ids)}'")


In [ ]:
import torch

def create_batches_sentencepiece_streaming(dataset, sp, config, max_batches=None):
    """Create batches from streaming dataset"""
    batch_count = 0
    epoch = 0
    pad_id = sp.pad_id() if hasattr(sp, 'pad_id') else 0

    while True:
        epoch += 1
        if epoch > 1:
            print(f"\n📚 Epoch {epoch}")

        batch_texts = []

        try:
            for sample in dataset:
                # Auto-detect text field
                text = None
                for field in ['text', 'content', 'document', 'body', 'article']:
                    if field in sample:
                        text = sample[field]
                        break

                if not text or len(str(text).strip()) < 20:
                    continue

                batch_texts.append(str(text))

                if len(batch_texts) >= config.batch_size:
                    encoded_batch = []
                    for t in batch_texts:
                        try:
                            token_ids = sp.encode(t, out_type=int)
                            if len(token_ids) > config.max_seq_len:
                                token_ids = token_ids[:config.max_seq_len]
                            pad_len = config.max_seq_len - len(token_ids)
                            token_ids = token_ids + [pad_id] * pad_len
                            encoded_batch.append(token_ids)
                        except:
                            continue

                    if len(encoded_batch) >= config.batch_size:
                        encoded_batch = encoded_batch[:config.batch_size]

                        input_ids = torch.tensor(encoded_batch, dtype=torch.long)
                        labels = input_ids.clone()
                        prosody = torch.rand(config.batch_size, config.max_seq_len, 4)
                        attention_mask = (input_ids != pad_id).long()
                        labels[attention_mask == 0] = -100

                        batch_count += 1
                        yield input_ids, labels, prosody, attention_mask

                        if max_batches and batch_count >= max_batches:
                            return

                        if batch_count % 100 == 0:
                            print(f"  Batches: {batch_count}")

                    batch_texts = []

        except Exception as e:
            print(f"  Dataset iteration ended: {e}")
            if epoch > 50:
                return


print("✅ Data loader function defined")


In [ ]:
import numpy as np
import torch.serialization

# Allow numpy dtype for checkpoint loading
torch.serialization.add_safe_globals([np.dtype])

def save_checkpoint(model, optimizer, scheduler, hippocampus, trainer,
                   global_step, losses, perplexities, steps, config):
    """Save checkpoint to Drive"""
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f'checkpoint_step_{global_step}.pt')

    try:
        checkpoint = {
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'global_step': global_step,
            'losses': losses,
            'perplexities': perplexities,
            'steps': steps,
            'hippocampus_memories': len(hippocampus.episodic_memories),
            'replay_buffer_size': len(trainer.replay_buffer),
            'config': config.__dict__,
        }

        torch.save(checkpoint, checkpoint_path)
        print(f"✅ Checkpoint saved: step_{global_step}")

        latest_path = os.path.join(CHECKPOINT_DIR, 'checkpoint_latest.pt')
        torch.save(checkpoint, latest_path)

        return True
    except Exception as e:
        print(f"❌ Checkpoint save failed: {e}")
        return False


def load_checkpoint(checkpoint_path, model, optimizer, scheduler):
    """Load checkpoint from Drive (PyTorch 2.6+ compatible)"""
    try:
        # Allow numpy types for checkpoint loading
        with torch.serialization.safe_globals([np.dtype]):
            checkpoint = torch.load(checkpoint_path, weights_only=False)

        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

        global_step = checkpoint['global_step']
        losses = checkpoint['losses']
        perplexities = checkpoint['perplexities']
        steps = checkpoint['steps']

        print(f"✅ Checkpoint loaded from step {global_step}")
        if perplexities:
            print(f"   Latest PPL: {perplexities[-1]:.2f}")

        return global_step, losses, perplexities, steps

    except Exception as e:
        print(f"⚠️ Checkpoint load failed: {e}")
        print(f"   Trying alternative loading method...")
        try:
            # Fallback: load with weights_only=False (less secure but works)
            checkpoint = torch.load(checkpoint_path, weights_only=False)

            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

            global_step = checkpoint['global_step']
            losses = checkpoint['losses']
            perplexities = checkpoint['perplexities']
            steps = checkpoint['steps']

            print(f"✅ Checkpoint loaded (fallback method) from step {global_step}")
            if perplexities:
                print(f"   Latest PPL: {perplexities[-1]:.2f}")

            return global_step, losses, perplexities, steps
        except Exception as e2:
            print(f"❌ All checkpoint loading failed: {e2}")
            return 0, [], [], []


print("✅ Checkpoint functions updated (PyTorch 2.6+ compatible)")


In [ ]:
print("\n" + "="*70)
print("Importing AURA modules...")
print("="*70)

try:
    from src.core.hippocampal import HippocampalFormation
    from src.core.language_zone.hippocampal_transformer import HippocampalTransformer
    from src.training.hippocampal_trainer import HippocampalTransformerTrainer
    print("✅ Imported AURA modules")
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("Make sure AURA source code is in /content or installed")
    raise

print("\n" + "="*70)
print("Initializing model...")
print("="*70)

gc.collect()
torch.cuda.empty_cache()

# Create hippocampus
hippocampus = HippocampalFormation(
    config.embedding_dim,
    config.n_place_cells,
    config.n_time_cells,
    config.n_grid_cells
)
print(f"✅ Hippocampus initialized")
print(f"   Place cells: {config.n_place_cells}")
print(f"   Time cells: {config.n_time_cells}")
print(f"   Grid cells: {config.n_grid_cells}")

# Create transformer
model = HippocampalTransformer(config, hippocampus)
model = model.to(device=device, dtype=torch.bfloat16)
print(f"✅ HippocampalTransformer initialized")

# Create trainer
trainer = HippocampalTransformerTrainer(model, config, hippocampus)
print(f"✅ Trainer initialized")

# Create optimizer
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config.lr,
    weight_decay=config.weight_decay,
    betas=(0.9, 0.95)
)
print(f"✅ Optimizer created")

# Create scheduler
def warmup_cosine(step):
    if step < config.warmup_steps:
        return (step + 1) / config.warmup_steps
    progress = (step - config.warmup_steps) / max(1, config.max_steps - config.warmup_steps)
    return 0.5 * (1 + np.cos(np.pi * progress))

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, warmup_cosine)
print(f"✅ Scheduler created")

total_params = sum(p.numel() for p in model.parameters())
print(f"\n📊 Model Statistics:")
print(f"   Parameters: {total_params / 1e6:.0f}M")
print(f"   GPU Memory: {torch.cuda.memory_allocated()/1e9:.2f}GB / {torch.cuda.get_device_properties(0).total_memory/1e9:.1f}GB")


After running the cell above, you will have updated the `hippocampal_trainer.py` file. To ensure these changes are loaded, please **re-run the following cells in your notebook**:

1.  **Cell `2B8vZeDjXU2N`**: To re-import the updated modules.
2.  **Cell `crnDjKa6bKQH`**: To re-initialize the `model`, `trainer`, `optimizer`, and `scheduler` with the corrected `EWCConsolidator` instantiation.
3.  **Cell `ZmGnMFqonGBj`**: To resume your training with the applied fix.

In [ ]:
#
# Recreate optimizer with new batch size
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config.lr,
    weight_decay=config.weight_decay,
    betas=(0.9, 0.95)
)
print("✅ Optimizer recreated")


In [ ]:
gc.collect()
torch.cuda.empty_cache()

print("Testing new batch size...")

test_input = torch.randint(0, config.vocab_size, (config.batch_size, config.max_seq_len)).to(device)
test_prosody = torch.zeros(config.batch_size, config.max_seq_len, 4, dtype=torch.bfloat16, device=device)

with autocast('cuda', dtype=torch.bfloat16):
    logits, _ = model(test_input, prosody=test_prosody, use_memory=True)
    loss = nn.CrossEntropyLoss()(logits.view(-1, config.vocab_size), test_input.view(-1))

loss.backward()

mem_used = torch.cuda.memory_allocated() / 1e9
mem_total = torch.cuda.get_device_properties(0).total_memory / 1e9
mem_pct = (mem_used / mem_total) * 100

print(f"✅ Test passed!")
print(f"   VRAM used: {mem_used:.2f}GB / {mem_total:.1f}GB ({mem_pct:.0f}%)")
print(f"   Headroom: {mem_total - mem_used:.2f}GB")

if mem_pct > 90:
    print("⚠️ Too high! Reduce batch_size")
elif mem_pct < 50:
    print("💡 Can increase batch_size more!")
else:
    print("✅ Optimal utilization!")

del test_input, test_prosody, logits, loss
optimizer.zero_grad()
gc.collect()
torch.cuda.empty_cache()


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Assuming these are defined elsewhere or imported
# from src.core.language_zone.hippocampal_transformer import HippocampalTransformer
# from src.core.hippocampal import HippocampalFormation

class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []

    def add(self, input_ids, labels, loss):
        if len(self.buffer) >= self.capacity:
            self.buffer.pop(0)
        self.buffer.append((input_ids, labels, loss))

    def sample(self, batch_size):
        if len(self.buffer) < batch_size:
            return []
        indices = torch.randint(len(self.buffer), (batch_size,)).tolist()
        return [self.buffer[i] for i in indices]

    def __len__(self):
        return len(self.buffer)


class EWCConsolidator:
    def __init__(self, model):
        self.model = model
        self.fisher = {}
        self.optpar = {}

    def compute_fisher(self, dataloader, device):
        original_dtype = next(self.model.parameters()).dtype
        # Temporarily convert model to float32 for Fisher calculation if not already
        if original_dtype != torch.float32:
            self.model.float()

        self.model.eval() # Set model to evaluation mode
        fisher_accumulator = {}
        for n, p in self.model.named_parameters():
            if p.requires_grad:
                fisher_accumulator[n] = torch.zeros_like(p.data, dtype=torch.float32)
                self.optpar[n] = p.data.clone().float()

        config = self.model.config

        for input_ids, labels in dataloader:
            input_ids = input_ids.to(device)
            labels = labels.to(device)

            # Create prosody. Input to model needs to be float32 for EWC pass.
            prosody = torch.rand(input_ids.size(0), config.max_seq_len, 4, dtype=torch.float32, device=device)
            # The HippocampalTransformer.forward() does not accept 'attention_mask'
            # attention_mask = (input_ids != 0).long().to(device) # Assuming 0 is pad_token_id

            self.model.zero_grad()
            with torch.enable_grad(): # Ensure gradients are enabled for Fisher calculation
                # FIX: Removed 'attention_mask' argument
                logits, _ = self.model(input_ids, prosody=prosody, use_memory=False)

                loss = nn.CrossEntropyLoss(reduction='mean')(
                    logits.view(-1, config.vocab_size),
                    labels.view(-1)
                )
            loss.backward()

            for n, p in self.model.named_parameters():
                if p.grad is not None:
                    fisher_accumulator[n] += p.grad.data.pow(2)

        # Average fisher over the dataloader size
        for n, f in fisher_accumulator.items():
            self.fisher[n] = f / len(dataloader)

        # Restore model to original dtype if it was changed
        if original_dtype != torch.float32:
            self.model.to(dtype=original_dtype)
        self.model.train() # Set model back to training mode


class HippocampalTransformerTrainer:
    def __init__(self, model, config, hippocampus):
        self.model = model
        self.config = config
        self.hippocampus = hippocampus
        self.optimizer = None # This will be passed from the training script
        self.scheduler = None # This will be passed from the training script

        self.global_step = 0
        self.losses = []
        self.perplexities = []
        self.steps = []

        self.replay_buffer = ReplayBuffer(capacity=getattr(config, 'replay_buffer_size', 50000))
        # FIX: Removed lambda_ewc argument from EWCConsolidator instantiation
        self.ewc = EWCConsolidator(model)

        self.phase = "wake"
        self.sleep_counter = 0

    def step_counter(self):
        self.global_step += 1
        if self.global_step % self.config.sleep_interval == 0:
            self.phase = "sleep"

    def train_step(self, input_ids, labels, prosody, attention_mask):
        # This method is typically called within the main training loop
        # and uses autocast, which is fine.
        pass # Actual training logic is in the main script for flexibility

    def consolidate(self, device):
        # This method is called during the sleep phase
        # EWC is used here, ewc_lambda comes from config
        if self.config.use_ewc and len(self.ewc.fisher) > 0:
            ewc_loss = 0
            for n, p in self.model.named_parameters():
                if n in self.ewc.fisher:
                    ewc_loss += (self.ewc.fisher[n] * (p - self.ewc.optpar[n])**2).sum()
            return ewc_loss * self.config.ewc_lambda # Use config.ewc_lambda here
        return 0.0


# Helper for plotting - assuming it's used elsewhere for visualizations
def plot_metrics(losses, perplexities, steps):
    # This is a placeholder, actual plotting would depend on matplotlib/seaborn etc.
    print("Plotting functionality not implemented in trainer.py")


After running the cell above, you will have updated the `hippocampal_trainer.py` file. To ensure these changes are loaded, please **re-run the following cells in your notebook**:

1.  **Cell `2B8vZeDjXU2N`**: To re-import the updated modules.
2.  **Cell `crnDjKa6bKQH`**: To re-initialize the `model`, `trainer`, `optimizer`, and `scheduler` with the corrected `EWCConsolidator` instantiation.
3.  **Cell `ZmGnMFqonGBj`**: To resume your training with the applied fix.

In [ ]:
import threading
import time
from datetime import datetime

# Stop any existing monitor
monitor_running = False
global_step = 0

def background_monitor():
    """Run monitoring in background thread"""
    global monitor_running
    monitor_running = True

    while monitor_running and global_step < config.max_steps:
        try:
            # Get current state
            step = global_step
            loss = losses[-1] if losses else 0
            ppl = perplexities[-1] if perplexities else 0
            best_ppl = min(perplexities) if perplexities else 0
            mem_count = len(hippocampus.episodic_memories)
            buf_size = len(trainer.replay_buffer)
            phase = trainer.phase

            # Calculate ETA
            if step > 0:
                eta_hours = (config.max_steps - step) / 66 / 60
            else:
                eta_hours = 0

            # Print status
            timestamp = datetime.now().strftime("%H:%M:%S")
            print(f"\n[{timestamp}] Step: {step:,}/50k | Loss: {loss:.3f} | PPL: {ppl:.2f} | Best: {best_ppl:.2f}")
            print(f"           Mem: {mem_count} | Buf: {buf_size:,} | Phase: {phase} | ETA: {eta_hours:.1f}h")

            time.sleep(60)  # Check every 60 seconds

        except Exception as e:
            print(f"Monitor error: {e}")
            time.sleep(60)

    print("\n✅ Monitor finished")

# Start background thread
monitor_thread = threading.Thread(target=background_monitor, daemon=True)
monitor_thread.start()

print("✅ Background monitor started (checks every 60 seconds)")
print("   Training will continue in parallel")


In [25]:
# CELL D1: Diagnose the issue
print("="*70)
print("DIAGNOSING REPETITION ISSUE")
print("="*70)

# Check what the model is actually outputting
model.eval()
test_prompt = "The history of"
test_ids = sp.encode(test_prompt, out_type=int)
input_ids = torch.tensor([test_ids], dtype=torch.long).to(device)

with torch.no_grad():
    with autocast('cuda', dtype=torch.bfloat16):
        prosody = torch.randn(1, len(test_ids), 4, dtype=torch.bfloat16, device=device)
        logits, _ = model(input_ids, prosody=prosody, use_memory=True)

        # Check the probability distribution
        probs = torch.softmax(logits[0, -1, :], dim=-1)
        top_k_probs, top_k_indices = torch.topk(probs, 10)

        print("\nTop 10 predictions for next token:")
        for prob, idx in zip(top_k_probs, top_k_indices):
            token_str = sp.id_to_piece(idx.item())
            print(f"  {token_str:20} : {prob.item():.4f}")

        # Check entropy
        entropy = -torch.sum(probs * torch.log(probs + 1e-10))
        print(f"\nEntropy: {entropy.item():.2f} (should be 2-5)")
        print(f"Max prob: {probs.max().item():.4f} (should be < 0.5)")

        if probs.max().item() > 0.8:
            print("\n⚠️ WARNING: Model is outputting one token with >80% probability!")
            print("   This causes repetition. The model may be underfitting or")
            print("   the learning rate might be too high causing instability.")

model.train()


DIAGNOSING REPETITION ISSUE

Top 10 predictions for next token:
  ▁of                  : 0.8787
  ▁the                 : 0.0006
  ▁                    : 0.0004
  s                    : 0.0004
  ▁is                  : 0.0003
  ▁for                 : 0.0003
  ▁to                  : 0.0002
  ,                    : 0.0002
  ▁are                 : 0.0002
  ▁it                  : 0.0002

Entropy: 1.60 (should be 2-5)
Max prob: 0.8787 (should be < 0.5)

⚠️ WARNING: Model is outputting one token with >80% probability!
   This causes repetition. The model may be underfitting or
   the learning rate might be too high causing instability.


HippocampalTransformer(
  (pos_encoder): ThetaGammaPositionalEncoding(embedding_dim=768, theta_freq=8.0Hz, gamma_freq=40.0Hz, freq_ratio=5.0)
  (semantic_encoder): PlaceCellSemanticEncoder(
    vocab_size=32000, embedding_dim=768, n_place_cells=2000, sparsity=3.0% (k=60)
    (token_embedding): Embedding(32000, 768)
    (semantic_projection): Linear(in_features=768, out_features=2000, bias=True)
    (place_to_semantic): Linear(in_features=2000, out_features=768, bias=True)
  )
  (dropout): Dropout(p=0.15, inplace=False)
  (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (layers): ModuleList(
    (0-11): 12 x HippocampalTransformerLayer(
      (attention_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attention): HippocampalProsodyAttention(
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=Tr

In [24]:
# The problem: softmax is collapsing to near-zero for all but one token
# Solution: Better numerical stability + prevent degenerate distributions

print("="*70)
print("FIXING NUMERICAL STABILITY")
print("="*70)

# Check if this is a training data issue
print(f"\nCurrent training stats:")
print(f"  Step: {global_step}")
print(f"  Loss: {losses[-1]:.3f}")
print(f"  PPL: {math.exp(min(losses[-1], 20)):.2f}")
print(f"  Eval PPL: {perplexities[-1] if perplexities else 'N/A'}")

# The model learned that "The history of" → "of" is common
# This is actually partially correct! But the issue is:
# 1. Softmax is numerically unstable (other tokens are exactly 0.0000)
# 2. Need to add eps to prevent this

print("\n⚠️ DIAGNOSIS:")
print("  • Model learned that after 'The history of' → next word is 'of'")
print("  • This is in the training data ('The history of history')")
print("  • But softmax collapsed all other tokens to exactly 0")
print("  • This causes generation to repeat the same token")

print("\n💡 SOLUTION:")
print("  • Reduce model size (12 layers might be overcapacity for 2200 steps)")
print("  • Or continue training - by step 10000+, model will learn better")
print("  • Or lower learning rate to prevent oscillation")

# For now, add numerical stability to generation
print("\n" + "="*70)


FIXING NUMERICAL STABILITY

Current training stats:
  Step: 11678
  Loss: 1.448
  PPL: 4.25
  Eval PPL: 1.1504640644594604

⚠️ DIAGNOSIS:
  • Model learned that after 'The history of' → next word is 'of'
  • This is in the training data ('The history of history')
  • But softmax collapsed all other tokens to exactly 0
  • This causes generation to repeat the same token

💡 SOLUTION:
  • Reduce model size (12 layers might be overcapacity for 2200 steps)
  • Or continue training - by step 10000+, model will learn better
  • Or lower learning rate to prevent oscillation



In [26]:
import threading
import time
from datetime import datetime

generation_running = False

def generate_text_stable(prompt, max_tokens=50, temperature=1.0):
    """Generate with numerical stability"""
    try:
        model.eval()

        token_ids = sp.encode(prompt, out_type=int)
        input_ids = torch.tensor([token_ids], dtype=torch.long).to(device)
        generated_tokens = list(token_ids)

        with torch.no_grad():
            with autocast('cuda', dtype=torch.bfloat16):
                for step in range(max_tokens):
                    if input_ids.shape[1] > config.max_seq_len:
                        input_ids = input_ids[:, -config.max_seq_len:]

                    prosody = torch.randn(1, input_ids.shape[1], 4, dtype=torch.bfloat16, device=device)
                    logits, _ = model(input_ids, prosody=prosody, use_memory=True)
                    logits = logits[0, -1, :].float()

                    # ===== NUMERICAL STABILITY FIX =====
                    # Subtract max to prevent overflow
                    logits = logits - logits.max()

                    # Apply temperature
                    logits = logits / temperature

                    # Convert to probabilities with numerical stability
                    # Use log_softmax to prevent underflow
                    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
                    probs = torch.exp(log_probs)

                    # Add small epsilon to prevent exact zeros
                    probs = probs + 1e-10
                    probs = probs / probs.sum()

                    # ===== BLOCK LAST 5 TOKENS =====
                    for token in generated_tokens[-5:]:
                        probs[token] = 1e-10
                    probs = probs / probs.sum()

                    # Sample
                    next_token = torch.multinomial(probs, 1)[0]

                    if next_token.item() == sp.eos_id():
                        break

                    generated_tokens.append(next_token.item())
                    input_ids = torch.cat([input_ids, next_token.unsqueeze(0).unsqueeze(0)], dim=1)

        result = sp.decode(generated_tokens)
        model.train()
        return result
    except Exception as e:
        return f"[Error]"


def background_generation_monitor():
    """Monitor generation"""
    global generation_running
    generation_running = True

prompts = [
        "The history of",
        "In the future",
        "Neural networks",
        "Machine learning",
        "Deep learning"
]


for p in prompts:
                gen = generate_text_stable(p, max_tokens=25, temperature=0.8)
                print(f"  '{p}' → '{gen[:70]}...'")

print("✅ Numerically Stable Generation Monitor")


  'The history of' → 'The history of Ingredient W stage Evolution blondeberry Ele Effectplea...'
  'In the future' → 'In the future ständig rug Shot frustrating resurse qualitiesblin Flexi...'
  'Neural networks' → 'Neural networks comparison aggregate Tab stalk Cash digestion Perhaps ...'
  'Machine learning' → 'Machine learning Bangladesh Joshua Numbershi diverizer son Practicalas...'
  'Deep learning' → 'Deep learning pipesProf density entsteht TopicBR needlecriticalinscrit...'
✅ Numerically Stable Generation Monitor


In [ ]:
# ============================================================================
# ADVANCED OPTIMIZATION METHODS
# ============================================================================

print("="*70)
print("ADDING OPTIMIZATION METHODS")
print("="*70)

# 1. Update config with regularization
config.weight_decay = 0.1        # L2 regularization (already in AdamW)
config.label_smoothing = 0.1     # Reduce from 0.2 (was too aggressive)
config.dropout = 0.1             # Reduce dropout (was 0.15)
config.gradient_clip = 1.0       # Gradient clipping

# 2. Add new optimization flags
config.use_cosine_annealing = True
config.use_gradient_checkpointing = False  # Enable if OOM
config.warmup_ratio = 0.06       # 6% warmup

print(f"✅ Config updated:")
print(f"   Weight decay (L2): {config.weight_decay}")
print(f"   Label smoothing: {config.label_smoothing}")
print(f"   Dropout: {config.dropout}")
print(f"   Gradient clip: {config.gradient_clip}")


# 3. Create optimized optimizer with weight decay groups
def create_optimizer_with_weight_decay(model, lr, weight_decay):
    """
    Separate parameters into groups:
    - With weight decay: Linear layers (L2 regularization)
    - Without weight decay: LayerNorm, biases, embeddings
    """
    decay_params = []
    no_decay_params = []

    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue

        # No weight decay on: biases, LayerNorm, embeddings
        if 'bias' in name or 'norm' in name or 'embedding' in name:
            no_decay_params.append(param)
        else:
            decay_params.append(param)

    optimizer_groups = [
        {'params': decay_params, 'weight_decay': weight_decay},
        {'params': no_decay_params, 'weight_decay': 0.0}
    ]

    optimizer = torch.optim.AdamW(
        optimizer_groups,
        lr=lr,
        betas=(0.9, 0.95),
        eps=1e-8
    )

    print(f"   Decay params: {len(decay_params)} tensors")
    print(f"   No-decay params: {len(no_decay_params)} tensors")

    return optimizer


# 4. Create new optimizer
print(f"\n🔧 Creating optimized AdamW with L2 regularization...")
optimizer = create_optimizer_with_weight_decay(model, config.lr, config.weight_decay)
print(f"✅ Optimizer created")


# 5. Warmup + Cosine Annealing with Restarts
def warmup_cosine_with_min_lr(step, warmup_steps, max_steps, min_lr_ratio=0.1):
    """
    Warmup + Cosine decay with minimum LR floor
    """
    if step < warmup_steps:
        return (step + 1) / warmup_steps

    progress = (step - warmup_steps) / max(1, max_steps - warmup_steps)
    cosine_decay = 0.5 * (1 + math.cos(math.pi * progress))

    # Floor at min_lr_ratio (10% of peak)
    return min_lr_ratio + (1 - min_lr_ratio) * cosine_decay


scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer,
    lambda step: warmup_cosine_with_min_lr(step, config.warmup_steps, config.max_steps, min_lr_ratio=0.1)
)

# Step to current position
for _ in range(global_step):
    scheduler.step()

print(f"✅ Scheduler created with LR floor at 10%")
print(f"   Current LR: {scheduler.get_last_lr()[0]:.2e}")

print("="*70)


In [ ]:
# Optional: Stochastic Weight Averaging for better generalization
from torch.optim.swa_utils import AveragedModel, SWALR

# Enable SWA after 20% of training
swa_start_step = int(config.max_steps * 0.2)
use_swa = True

if use_swa:
    swa_model = AveragedModel(model)
    swa_scheduler = SWALR(optimizer, swa_lr=1e-5)
    print(f"✅ SWA enabled (starts at step {swa_start_step})")
else:
    swa_model = None
    print("⚠️ SWA disabled")


In [21]:

print("="*70)
print("🚀 OPTIMIZED L4 CONFIG")
print("="*70)
print(f"Batch size: {config.batch_size} (4x increase)")
print(f"Gradient accumulation: {config.gradient_accumulation}")
print(f"Effective batch: {config.batch_size * config.gradient_accumulation}")
print(f"Replay buffer: {config.replay_buffer_size}")
print(f"Expected VRAM: ~16-18GB")
print("="*70 + "\n")

optimizer.zero_grad()
gc.collect()
torch.cuda.empty_cache()



latest_checkpoint = os.path.join(CHECKPOINT_DIR, 'checkpoint_step_2000.pt')


# Reload checkpoint with new optimizer
global_step, losses, perplexities, steps = load_checkpoint(
    latest_checkpoint, model, optimizer, scheduler
)

print(f"✅ Resumed from step {global_step}")
print(f"   Batch: {config.batch_size} × {config.gradient_accumulation}")

# ===== START OPTIMIZED TRAINING =====
print("\n" + "="*70)
print("🚀 RESUMING TRAINING (OPTIMIZED)")
print("="*70)

accumulation_step = 0
train_gen = create_batches_sentencepiece_streaming(
    dataset, sp, config,
    max_batches=(config.max_steps - global_step) * config.gradient_accumulation
)

pbar = tqdm(total=config.max_steps - global_step, desc="Training (Optimized)")
model.train()
start_time = time.time()

try:
    for input_ids, labels, prosody, attention_mask in train_gen:
        accumulation_step += 1

        input_ids = input_ids.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        prosody = prosody.to(device, dtype=torch.bfloat16, non_blocking=True)

        # ===== WAKE PHASE =====
        if trainer.phase == "wake":
            try:
                with autocast('cuda', dtype=torch.bfloat16):
                    logits, place_cell_activity = model(input_ids, prosody=prosody, use_memory=True)
                    loss = nn.CrossEntropyLoss(label_smoothing=config.label_smoothing)(
                        logits.view(-1, config.vocab_size),
                        labels.view(-1)
                    )

               # Gradient accumulation
                scaled_loss = loss / config.gradient_accumulation
                scaled_loss.backward()

                if accumulation_step % config.gradient_accumulation == 0:
                    global_step += 1
                    trainer.step_counter()

                    # ===== GRADIENT CLIPPING =====
                    grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), config.gradient_clip)

                    optimizer.step()

                    # ===== SWA UPDATE (if enabled) =====
                    if use_swa and global_step >= swa_start_step:
                        swa_model.update_parameters(model)
                        swa_scheduler.step()
                    else:
                        scheduler.step()

                    optimizer.zero_grad(set_to_none=True)

                    # Track gradient norm for debugging
                    if global_step % 100 == 0:
                        print(f"   Grad norm: {grad_norm:.2f}")
                    losses.append(loss.item())

                    # Store in replay buffer
                    if global_step % 2 == 0:
                        trainer.replay_buffer.add(
                            input_ids.detach().cpu(),
                            labels.detach().cpu(),
                            loss.item()
                        )

                    elapsed = time.time() - start_time
                    speed = global_step / elapsed if elapsed > 0 else 0
                    current_lr = scheduler.get_last_lr()[0]
                    mem_used = torch.cuda.memory_allocated() / 1e9

                    pbar.set_postfix({
                        'loss': f"{loss.item():.3f}",
                        'lr': f"{current_lr:.2e}",
                        'it/s': f"{speed:.2f}",
                        'mem': f"{mem_used:.1f}GB",
                        'mem_count': len(hippocampus.episodic_memories)
                    })

                    # ===== EPISODIC MEMORY CREATION =====
                    if global_step % config.memory_creation_interval == 0:
                        with torch.no_grad():
                            feats = place_cell_activity.float().mean(dim=0).cpu().numpy()

                        hippocampus.create_episodic_memory(
                            memory_id=f"step_{global_step}",
                            event_id=f"train_{global_step}",
                            features=feats,
                            associated_experts=None
                        )

                    # ===== EVALUATION =====
                    if global_step % config.eval_interval == 0:
                        model.eval()
                        eval_loss = 0
                        eval_count = 0

                        with torch.no_grad():
                            for sample in dataset:
                                if eval_count >= 20:
                                    break

                                text = None
                                for field in ['text', 'content', 'document', 'body']:
                                    if field in sample:
                                        text = sample[field]
                                        break

                                if not text or len(str(text).strip()) < 50:
                                    continue

                                try:
                                    token_ids = sp.encode(str(text), out_type=int)
                                    if len(token_ids) < 10:
                                        continue
                                    if len(token_ids) > config.max_seq_len:
                                        token_ids = token_ids[:config.max_seq_len]
                                    pad_len = config.max_seq_len - len(token_ids)
                                    token_ids = token_ids + [sp.pad_id()] * pad_len

                                    eval_input = torch.tensor([token_ids], dtype=torch.long).to(device)
                                    eval_labels = eval_input.clone()
                                    eval_labels[eval_input == sp.pad_id()] = -100
                                    eval_prosody = torch.zeros(1, config.max_seq_len, 4, dtype=torch.bfloat16, device=device)

                                    with autocast('cuda', dtype=torch.bfloat16):
                                        eval_logits, _ = model(eval_input, prosody=eval_prosody, use_memory=True)
                                        batch_loss = nn.CrossEntropyLoss()(
                                            eval_logits.view(-1, config.vocab_size),
                                            eval_labels.view(-1)
                                        )

                                    if not torch.isnan(batch_loss):
                                        eval_loss += batch_loss.item()
                                        eval_count += 1
                                except:
                                    continue

                        ppl = math.exp(min(eval_loss / max(eval_count, 1), 20))
                        perplexities.append(ppl)
                        steps.append(global_step)

                        train_ppl = math.exp(min(sum(losses[-50:])/min(len(losses),50), 20))
                        print(f"\n📊 Step {global_step}: Train PPL={train_ppl:.2f} | Eval PPL={ppl:.2f} | LR={current_lr:.2e}")
                        print(f"   VRAM: {mem_used:.1f}GB | Memories: {len(hippocampus.episodic_memories)} | Buffer: {len(trainer.replay_buffer)}")

                        model.train()

                    # ===== CHECKPOINTING =====
                    if global_step % 500 == 0:
                        save_checkpoint(model, optimizer, scheduler, hippocampus, trainer,
                                      global_step, losses, perplexities, steps, config)

                    if perplexities and perplexities[-1] == min(perplexities):
                        best_path = os.path.join(CHECKPOINT_DIR, 'checkpoint_best.pt')
                        torch.save({
                            'model_state_dict': model.state_dict(),
                            'global_step': global_step,
                            'ppl': perplexities[-1]
                        }, best_path)
                        print(f"🏆 Best: PPL={perplexities[-1]:.2f}")

                    pbar.update(1)

                    if global_step >= config.max_steps:
                        break

            except RuntimeError as e:
                if "out of memory" in str(e):
                    print(f"\n⚠️ OOM at step {global_step}")
                    print("Reduce batch_size in config")
                    optimizer.zero_grad(set_to_none=True)
                    gc.collect()
                    torch.cuda.empty_cache()
                    break
                else:
                    raise
            except Exception as e:
                print(f"\n❌ Error: {e}")
                traceback.print_exc()
                break

        # ===== SLEEP PHASE =====
        elif trainer.phase == "sleep":
            print(f"\n🌙 Sleep Phase at step {global_step} - Memory Consolidation")
            try:
                gc.collect()
                torch.cuda.empty_cache()

                # ===== FISHER INFORMATION COMPUTATION =====
                if not trainer.ewc.fisher and len(trainer.replay_buffer) > 0 and config.use_ewc:
                    print("  📍 Computing Fisher Information (Elastic Weight Consolidation)...")
                    try:
                        samples = trainer.replay_buffer.sample(min(20, len(trainer.replay_buffer)))
                        mock_loader = [(s[0].unsqueeze(0).to(device), s[1].unsqueeze(0).to(device)) for s in samples]

                        # ===== FIX: Convert to float32 for Fisher computation =====
                        model.float()  # Temporary conversion
                        trainer.ewc.compute_fisher(mock_loader, device=device)
                        model.to(dtype=torch.bfloat16)  # Convert back to bfloat16

                        print("  ✅ Fisher Information computed")
                    except Exception as e:
                        print(f"  ⚠️ Fisher computation skipped: {e}")
                        model.to(dtype=torch.bfloat16)  # Ensure model is back to bfloat16

                # ===== EXPERIENCE REPLAY WITH BACKWARD REPLAYS =====
                print(f"  🔄 Replaying {config.sleep_steps} batches from memory...")
                replay_count = 0

                for i in range(config.sleep_steps):
                    try:
                        samples = trainer.replay_buffer.sample(config.batch_size)
                        if not samples or len(samples) < 2:
                            continue

                        replay_in = torch.stack([s[0] for s in samples]).to(device)
                        replay_lab = torch.stack([s[1] for s in samples]).to(device)

                        # Backward replay: temporal reversal for memory consolidation
                        if i % 5 >= 3:
                            replay_in = torch.flip(replay_in, [0])

                        optimizer.zero_grad(set_to_none=True)

                        with autocast('cuda', dtype=torch.bfloat16):
                            out, _ = model(replay_in, use_memory=True)
                            r_loss = nn.CrossEntropyLoss(label_smoothing=config.label_smoothing)(
                                out.view(-1, config.vocab_size),
                                replay_lab.view(-1)
                            )

                        # Reduced learning for replay (0.1x)
                        (r_loss * 0.1).backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                        optimizer.step()
                        replay_count += 1
                    except:
                        continue

                print(f"  ✅ Replay complete: {replay_count}/{config.sleep_steps} batches")
                        # Memory Decay
                try:
                    hippocampus.decay_memories(decay_rate=config.memory_decay_rate)
                    print(f"  📉 Memory decay | Memories: {len(hippocampus.episodic_memories)}")
                except Exception as e:
                    print(f"  ⚠️ Decay failed: {e}")

                trainer.phase = "wake"
                gc.collect()
                torch.cuda.empty_cache()

            except Exception as e:
                print(f"\n❌ Sleep error: {e}")
                traceback.print_exc()
                trainer.phase = "wake"

except KeyboardInterrupt:
    print("\n⏹️ Interrupted")
    save_checkpoint(model, optimizer, scheduler, hippocampus, trainer,
                  global_step, losses, perplexities, steps, config)
    print("✅ Checkpoint saved")

except Exception as e:
    print(f"\n❌ Fatal: {e}")
    traceback.print_exc()
    save_checkpoint(model, optimizer, scheduler, hippocampus, trainer,
                  global_step, losses, perplexities, steps, config)

finally:
    pbar.close()
    elapsed_time = time.time() - start_time

    save_checkpoint(model, optimizer, scheduler, hippocampus, trainer,
                   global_step, losses, perplexities, steps, config)

    print("\n" + "="*70)
    print("✅ TRAINING COMPLETE (OPTIMIZED)")
    print("="*70)
    print(f"⏱️  Total Time: {elapsed_time/3600:.2f} hours")
    print(f"📊 Steps: {global_step}/{config.max_steps}")
    print(f"🚀 Speed: {global_step / elapsed_time:.2f} it/s")
    print(f"🧠 Memories: {len(hippocampus.episodic_memories)}")

    if perplexities:
        train_ppl = math.exp(min(sum(losses[-50:])/min(len(losses),50), 20))
        print(f"📈 Final Train PPL: {train_ppl:.2f}")
        print(f"📈 Final Eval PPL: {perplexities[-1]:.2f}")
        print(f"📈 Best PPL: {min(perplexities):.2f}")

    print(f"💾 Checkpoints: {CHECKPOINT_DIR}")
    print("="*70)


Training (Optimized):  12%|█▏        | 6059/50000 [2:44:34<18:04:51,  1.48s/it, loss=1.421, lr=2.94e-04, it/s=0.61, mem=3.7GB, mem_count=1211]


[07:23:38] Step: 6,060/50k | Loss: 1.421 | PPL: 1.17 | Best: 1.14
           Mem: 1212 | Buf: 48,480 | Phase: wake | ETA: 11.1h


Training (Optimized):  12%|█▏        | 6084/50000 [2:45:36<17:51:37,  1.46s/it, loss=1.450, lr=2.94e-04, it/s=0.61, mem=3.7GB, mem_count=1216]


[07:24:38] Step: 6,085/50k | Loss: 1.450 | PPL: 1.17 | Best: 1.14
           Mem: 1217 | Buf: 48,672 | Phase: wake | ETA: 11.1h


Training (Optimized):  12%|█▏        | 6097/50000 [2:46:11<27:59:31,  2.30s/it, loss=1.431, lr=2.94e-04, it/s=0.61, mem=3.7GB, mem_count=1219]

...

Training (Optimized):  23%|██▎       | 11678/50000 [10:17:23<51:51:07,  4.87s/it, loss=1.448, lr=2.80e-04, it/s=0.32, mem=3.7GB, mem_count=2335]


⏹️ Interrupted
✅ Checkpoint saved: step_11678


Training (Optimized):  23%|██▎       | 11678/50000 [10:17:28<33:46:16,  3.17s/it, loss=1.448, lr=2.80e-04, it/s=0.32, mem=3.7GB, mem_count=2335]

✅ Checkpoint saved


✅ Checkpoint saved: step_11678

✅ TRAINING COMPLETE (OPTIMIZED)
⏱️  Total Time: 10.29 hours
📊 Steps: 11678/50000
🚀 Speed: 0.32 it/s
🧠 Memories: 2335
📈 Final Train PPL: 4.29
📈 Final Eval PPL: 1.15
📈 Best PPL: 1.14
💾 Checkpoints: /content/drive/MyDrive/aura_checkpoints


In [ ]:
print("="*70)
print("DIAGNOSING PLATEAU / REGRESSION")
print("="*70)

print(f"\n📊 Current Status:")
print(f"   Step: {global_step}")
print(f"   Loss: {losses[-1]:.3f} (should be ~2.6)")
print(f"   Train PPL: {math.exp(min(losses[-1], 20)):.2f}")
print(f"   Eval PPL: {perplexities[-1]:.2f}")
print(f"   Best PPL: {min(perplexities):.2f}")
print(f"   LR: {scheduler.get_last_lr()[0]:.2e}")

# Check loss history
if len(losses) > 100:
    early_avg = sum(losses[:100]) / 100
    recent_avg = sum(losses[-100:]) / 100
    print(f"\n📉 Loss Trend:")
    print(f"   Early avg (first 100): {early_avg:.3f}")
    print(f"   Recent avg (last 100): {recent_avg:.3f}")

    if recent_avg > early_avg + 0.5:
        print(f"   ⚠️ WARNING: Loss INCREASED by {recent_avg - early_avg:.3f}")
        print(f"   This suggests training regressed!")

# Check if we're in the cosine decay part
warmup_done = global_step > config.warmup_steps
progress = (global_step - config.warmup_steps) / max(1, config.max_steps - config.warmup_steps)
print(f"\n⏱️ Schedule Progress:")
print(f"   Warmup complete: {warmup_done}")
print(f"   Cosine progress: {progress*100:.1f}%")
print(f"   Current LR: {scheduler.get_last_lr()[0]:.2e} (peak was {config.lr:.2e})")

print("="*70)


In [ ]:
print("="*70)
print("FIXING LEARNING RATE")
print("="*70)

print(f"\nCurrent LR: {scheduler.get_last_lr()[0]:.2e}")
print(f"Config peak LR: {config.lr:.2e}")
print(f"Issue: LR never warmed up to peak!")

config.lr = 3e-4

# Solution: Create fresh optimizer and scheduler at correct position
print(f"\n🔧 Resetting optimizer to step {global_step}...")

# Create new optimizer
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=3e-4,  # Start at config.lr (3e-4)
    weight_decay=config.weight_decay,
    betas=(0.9, 0.95)
)

# Create new scheduler
def warmup_cosine(step):
    if step < config.warmup_steps:
        return (step + 1) / config.warmup_steps
    progress = (step - config.warmup_steps) / max(1, config.max_steps - config.warmup_steps)
    return 0.5 * (1 + np.cos(np.pi * progress))

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, warmup_cosine)

# Step scheduler to current position
for _ in range(global_step):
    scheduler.step()

current_lr = scheduler.get_last_lr()[0]
print(f"✅ New optimizer created")
print(f"   Current LR: {current_lr:.2e}")
print(f"   Warmup steps: {config.warmup_steps}")
print(f"   At step {global_step} (warmup done: {global_step >= config.warmup_steps})")

print("="*70)
